# Solving the Double Pendulum Problem Using Euler-Lagrange

Defines a DoublePendulum class that constructs pendulum plots from solving the Euler-Lagrange equations.

By Karish Seebaluck (seebaluck.1@osu.edu)

In [1]:
%matplotlib inline

import numpy as np
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
from IPython.display import Image

From [Wolfram Research](https://scienceworld.wolfram.com/physics/DoublePendulum.html) we get the picture below. We will also adopt their conventions with $\hat{x}$ pointing to the right and $\hat{y}$ pointing up.

In [2]:
Image(url='https://scienceworld.wolfram.com/physics/dimg270.gif')

Our two generalized coordinates will be $\theta$<sub>1</sub> and $\theta$<sub>2</sub> and they can be seen in our image above. The positions and velocities for our pendulum will be given by,<p>
    
<center>$x_{1} = l_{1}sin(\theta_{1})\qquad\dot{x_{1}} = l_{1}\dot{\theta_{1}}cos(\theta_{1})$</center><p>
<center>$y_{1} = -l_{1}cos(\theta_{1})\qquad\dot{y_{1}} = l_{1}\dot{\theta_{1}}sin(\theta_{1})$</center><p>
<center>$x_{2} = l_{1}sin(\theta_{1}) + l_{2}sin(\theta_{2})\qquad\dot{x_{2}} = l_{1}\dot{\theta_{1}}cos(\theta_{1}) + l_{2}\dot{\theta_{2}}cos(\theta_{2})$</center><p>
<center>$y_{2} = -l_{1}cos(\theta_{1}) - l_{2}cos(\theta_{2})\qquad\dot{y_{2}} = l_{1}\dot{\theta_{1}}sin(\theta_{1}) + l_{2}\dot{\theta_{2}}sin(\theta_{2})$</center><p>
    
The potential, $U$, and kinetic, $T$, energies of our system will be given by,<p>
<center>$U = m_{1}gy_{1} + m_{2}gy_{2} = -(m_{1} + m_{2})gl_{1}cos(\theta_{1}) - m_{2}gl_{2}cos(\theta_{2})$<center><br>
<center>$T = \frac{1}{2}m_{1}v_{1}^{2} + \frac{1}{2}m_{2}v_{2}^{2}$<center><br>
<center>$ = \frac{1}{2}m_{1}l_{1}^{2}\dot{\theta_{1}^{2}} + \frac{1}{2}m_{2}[l_{1}^{2}\dot{\theta_{1}^{2}} + l_{2}^{2}\dot{\theta_{2}^{2}} + 2l_{1}l_{2}\dot{\theta_{1}}\dot{\theta_{2}}cos(\theta_{1} - \theta_{2})]$<center><p>

Then the Lagrangian, $L$, for our system is going to be given by,<br>
<center>$ L = T - U$<center><br>
<center>$ = \frac{1}{2}(m_{1} + m_{2})l_{1}^{2}\dot{\theta_{1}^{2}} + \frac{1}{2}m_{2}l_{2}^{2}\dot{\theta_{2}^{2}} + m_{2}l_{1}l_{2}\dot{\theta_{1}}\dot{\theta_{2}}cos(\theta_{1}-\theta_{2}) + (m_{1} + m_{2})gl_{1}cos(\theta_{1}) + m_{2}gl_{2}cos(\theta_{2})$<p>

The Euler-Lagrange equations are given by,<p>
<center>$\large \frac{d}{dt}(\frac{\partial L}{\partial \dot{q_{i}}}) - \frac{\partial L}{\partial \dot{q_{i}}} = 0 $<p>
    
After computing the derivates, we get the following differential equations,<p>
<center>$(m_{1}+m_{2})l_{1}\ddot{\theta_{1}} + m_{2}l_{2}\ddot{\theta_{2}}cos(\theta_{1} - \theta_{2}) + m_{2}l_{2}\dot{\theta_{2}^{2}}sin(\theta_{1} - \theta_{2}) + g(m_{1} + m_{2})sin(\theta_{1}) = 0$<p>
<center>$m_{2}l_{2}\ddot{\theta_{2}} + m_{2}l_{1}\ddot{\theta_{1}}cos(\theta_{1} - \theta_{2}) - m_{2}l_{1}\dot{\theta_{1}^{2}}sin(\theta_{1} - \theta_{2}) + m_{2}gsin(\theta_{2}) = 0$<p>
    
Scipy's solve_ivp function requires a system of first-order differential equations. Let $z_{1} = \dot{\theta_{1}}$ and $z_{2} = \dot{\theta_{2}}$, then $\dot{z_{1}} = \ddot{\theta_{1}}$ and $\dot{z_{2}} = \ddot{\theta_{2}}$. Rearranging our differential equations above, we get the following,<p>
<center> $\dot{z_{1}} = \large\frac{m_{2}gsin(\theta_{2})cos(\theta_{1} - \theta_{2}) - m_{2}sin(\theta_{1} - \theta_{2})[l_{1}z_{1}^{2}cos(\theta_{1} - \theta_{2}) + l_{2}z_{2}^{2}] - (m_{1} + m_{2})gsin(\theta_{1})}{l_{1}[m_{1} + m_{2}sin^{2}(\theta_{1} - \theta_{2})]}$<p>
<center> $\dot{z_{2}} = \large\frac{(m_{1} + m_{2})[l_{1}z_{1}^{2}sin(\theta_{1} - \theta_{2}) - gsin(\theta_{2}) + gsin(\theta_{1})cos(\theta_{1} - \theta_{2})] + m_{2}l_{2}z_{2}^{2}sin(\theta_{1} - \theta_{2})cos(\theta_{1} - \theta_{2})}{l_{2}[m_{1} + m_{2}sin^{2}(\theta_{1} - \theta_{2})]}$


In [3]:
class DoublePendulum():
    """
    This class takes in multiple parameters and the Lagrange equations to solve a simple double pendulum problem.
    
    Parameters
    ----------
    l1 : float
        This is the length of the top pendulum
    m1 : float
        The mass of the top pendulum
    l2 : float
        The length of the bottom pendulum
    m2 : float
        The mass of the bottom pendulum
    g  : float
        gravitation acceleration in m/s^2
    
    Methods
    -------
    dy_dt(t,y)
        Returns the right side of the differential equation in multiple component vector y,
        given time t and the corresponding value of y.
    """
    
    def __init__(self, L1=1., L2=1., m1=1., m2=1., g=1.):
        self.L1 = L1
        self.L2 = L2
        self.m1 = m1
        self.m2 = m2
        self.g = g
        
    def dy_dt(self, t, y):
        """
        Parameters
        ----------
        t : float
            time
        y : list with floats
            A 4-component vector with y[0] = phi1(t), y[1] = dphi1(t)/dt,
            y[2] = phi2(t), y[3] = dphi2(t)/dt
        
        Returns
        -------
        This function returns the right-hand side of the diff eq,
        [dphi1/dt, dphi1^2/dt^2, dphi2/dt, dphi2^2/dt^2]
        
        """
        finalvec = []
        finalvec.append(y[0])
        finalvec.append((self.m2 * self.g * np.sin(y[0]) * np.cos(y[0] - y[2]) - self.m2 * np.sin(y[0] - y[2]) * (self.L1 * y[1]**2 * \
                        np.cos(y[0] - y[2]) + self.L2 * y[3]**2) - (self.m1 + self.m2) * self.g * np.sin(y[0])) / \
                        (self.L1 * (self.m1 + self.m2 * np.sin(y[0] - y[2])**2)))
        finalvec.append(y[2])
        finalvec.append(((self.m1 + self.m2) * (self.L1 * y[1]**2 * np.sin(y[0] - y[2]) - self.g * np.sin(y[2]) + self.g \
                        * np.sin(y[0]) * np.cos(y[0] - y[2])) + self.m2 * self.L2 * y[3]**2 * np.sin(y[0] - y[2]) * \
                         np.cos(y[0] - y[2])) / (self.L2 * (self.m1 + self.m2 * np.sin(y[0] - y[2])**2)))
        return finalvec[0], finalvec[1], finalvec[2], finalvec[3]
    
    
    def solve_ode(self, t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0,
                  abserr=1.0e-10, relerr=1.0e-10):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi1_0, phi1_dot_0, phi2_0, phi2_dot_0]
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, atol=abserr, rtol=relerr)
        phi1, phi1_dot, phi2, phi2_dot = solution.y
        return phi1, phi1_dot, phi2, phi2_dot

In [4]:
# Initialize our pendulums
p1 = DoublePendulum(g=1,L1=1,L2=1,m1=1,m2=1)
phi1_0 = np.pi/2
phi1_dot_0 = 0
phi2_0 = 0
phi2_dot_0 = 0

t_start = 0.
t_end = 50.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)

# Solve for the motion of our pendulums
phi1, phi1_dot, phi2, phi2_dot = p1.solve_ode(t_pts,phi1_0, phi1_dot_0, phi2_0, phi2_dot_0)

# Making our figures
phi_vs_time_labels = (r'$t$', r'$\phi(t)$')

fig = plt.figure(figsize=(15,5))
overall_title = 'Simple Double pendulum from Lagrangian:  ' + \
                rf'  $\phi_1 = {phi1:.2f},$' + \
                rf' $\dot\phi_1 = {phi1_dot:.2f}$' + \
                rf'  $\phi_2 = {phi2:.2f},$' + \
                rf' $\dot\phi_2 = {phi2_dot:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi1 plot 
ax_a = fig.add_subplot(1,3,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=None, 
            title=r'$\phi(t)$', 
            ax=ax_a)  

KeyboardInterrupt: 